# Prompting Langchain

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*DkgQ23ng-8J2sl66_YQWnw.png">

In [1]:
from typing import List
!pip install langchain-openai
!pip install langchain
!pip install langchain_community
!pip install langchain_core
!pip install gradio
!pip install unstructured
!pip install unstructured[pdf]
!pip install Chromadb
!pip install numpy==1.24.1py
!pip install langdetect

zsh:1: no matches found: unstructured[pdf]
ERROR: Invalid requirement: 'numpy==1.24.1py': Expected end or semicolon (after version specifier)
    numpy==1.24.1py
         ~~~~~~~~^


In [1]:
import os

from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from operator import itemgetter
from langdetect import detect, detect_langs

In [19]:
# file = 'CRA023005V5-ASS28082024V01-000727267.pdf'
# file = '118498.pdf'
file = '573735.pdf'

encoding_name="cl100k_base"
path_faiss="/tmp/faiss_index"
deployment_name = "leitura_documento_gpt4o"
embedding_model = "text-embedding"
openai_api_version="2023-05-15"
chunk_size = 1024
chunk_overlap = 256

model = "gpt-4o"

verbose=False
temperature = 0
k = 5
fetch_k = 20

# Loading doc using Unstructured-IO

In [3]:
print("Loading Unstructured Data")
# modes: “single”, “elements”, and “paged”
# loader = UnstructuredFileLoader(pdf, mode="elements")
loader = UnstructuredFileLoader(file, mode="paged", strategy="fast")
raw_doc = loader.load()
raw_doc

Loading Unstructured Data


[Document(page_content='TERMO DE SECURITIZAÇÃO DE DIREITOS CREDITÓRIOS DO AGRONEGÓCIO\n\npara emissão de\n\nCERTIFICADOS DE RECEBÍVEIS DO AGRONEGÓCIO DA 1ª E 2ª SÉRIES DA 34ª (TRIGÉSIMA QUARTA) EMISSÃO DA\n\nCERES SECURITIZADORA S.A. como companhia securitizadora\n\nlastreados em direitos creditórios do agronegócio devidos pela NATIVA AGRONEGÓCIOS LTDA.\n\ncelebrado com\n\nOLIVEIRA TRUST DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A. como agente fiduciário\n\nDatado de 18 de dezembro de 2023\n\nDocumento assinado no Assinador Registro de Imóveis. Para validar o documento e suas assinaturas acesse https://assinador.registrodeimoveis.org.br/validate/546KQ-TBSM7-VTY3M-6YDA8.\n\n', metadata={'source': '573735.pdf', 'coordinates': {'points': ((590.85, 42.51999999999998), (590.85, 695.72), (605.03, 695.72), (605.03, 42.51999999999998)), 'system': 'PixelSpace', 'layout_width': 612.12, 'layout_height': 792.12}, 'filename': '573735.pdf', 'languages': ['eng'], 'last_modified': '2024-09-09T0

In [4]:
str_template_prompt_initial = \
"""
Responda à pergunta o mais detalhadamente possível a partir do contexto fornecido.
Certifique-se de fornecer todos os detalhes. Quebre as tarefas em pequenas tarefas e passos.
Caso a resposta não estiver no contexto fornecido, basta dizer "a resposta não está disponível no contexto".
Não forneça respostas erradas ou respostas que não estão no contexto fornecido.
Não coloque comentários dentro do JSON retornado para explicar os atributos.

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:
SOURCES:
"""

# Splitting doc in chuncks using RecursiveCharacterTextSplitter

In [5]:
# segmenting the document into segments
text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", " ", ""])
chunk_docs = text_splitter.split_documents(raw_doc)

In [6]:
chunk_docs[0]

Document(page_content='TERMO DE SECURITIZAÇÃO DE DIREITOS CREDITÓRIOS DO AGRONEGÓCIO\n\npara emissão de\n\nCERTIFICADOS DE RECEBÍVEIS DO AGRONEGÓCIO DA 1ª E 2ª SÉRIES DA 34ª (TRIGÉSIMA QUARTA) EMISSÃO DA\n\nCERES SECURITIZADORA S.A. como companhia securitizadora\n\nlastreados em direitos creditórios do agronegócio devidos pela NATIVA AGRONEGÓCIOS LTDA.\n\ncelebrado com\n\nOLIVEIRA TRUST DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A. como agente fiduciário\n\nDatado de 18 de dezembro de 2023\n\nDocumento assinado no Assinador Registro de Imóveis. Para validar o documento e suas assinaturas acesse https://assinador.registrodeimoveis.org.br/validate/546KQ-TBSM7-VTY3M-6YDA8.', metadata={'source': '573735.pdf', 'coordinates': {'points': ((590.85, 42.51999999999998), (590.85, 695.72), (605.03, 695.72), (605.03, 42.51999999999998)), 'system': 'PixelSpace', 'layout_width': 612.12, 'layout_height': 792.12}, 'filename': '573735.pdf', 'languages': ['eng'], 'last_modified': '2024-09-09T08:37:

# Get Embeddings

In [7]:
embeddings = AzureOpenAIEmbeddings(model=embedding_model)

# Saving chunks in local database

In [9]:
vector_store = FAISS.from_documents(chunk_docs, embedding=embeddings)
vector_store.save_local(path_faiss)

# Creating Prompt

In [10]:
def get_schema(campo):
    schema = ResponseSchema(
        name=campo[0],
        type=campo[1],
        description=campo[2]
    )

    return schema

In [11]:
def create_user_question(campos):
    nomes_campos = [campo[0] for campo in campos]
    
    response_schema = [
        get_schema(campo) for campo in campos
    ]
    
    output_parser = StructuredOutputParser.from_response_schemas(response_schema)
    schema_formatado = output_parser.get_format_instructions()
    
    template = PromptTemplate.from_template(
    """
    Extraia as seguintes informações:
    
    {nomes_campos}
    
    {schema}
    """, partial_variables={'nomes_campos': ', '.join(nomes_campos) ,'schema': schema_formatado})
    
    user_question = template.format()
    print(user_question)
    return output_parser, user_question

# Retrieving relevants data

In [20]:
def get_docs_relevants(user_question):
    vector_store_loaded = FAISS.load_local(path_faiss, embeddings, allow_dangerous_deserialization=True)
    
    docs_relevants = vector_store_loaded.similarity_search(user_question, k=k, fetch_k=fetch_k)
    return docs_relevants

# Question to LLM

In [21]:
def invoke_llm(user_question, docs_relevants):
    template_prompt_initial = PromptTemplate.from_template(str_template_prompt_initial)
    
    # llm = AzureChatOpenAI(deployment_name=deployment_name, model_name=model, temperature=temperature)
    llm = AzureChatOpenAI(azure_deployment=deployment_name, openai_api_version=openai_api_version,)
    
    chain = load_qa_with_sources_chain(llm=llm, chain_type="stuff", prompt=template_prompt_initial, verbose=verbose)
    
    input_msg = {"input_documents": docs_relevants, "question": user_question}
    
    response = chain.invoke(input_msg, return_only_outputs=True,)

    if "output_text" in response:        
        return response["output_text"]
    else:
        print("Error")
    return None

# Executing LLM

In [43]:
from dataclasses import dataclass

@dataclass
class CamposLLM:
    """Class for keeping track of an item in inventory."""
    name: str
    type: str
    description: str
    value: str = ""
    depends_on = []

In [23]:
# campos = [
#     ("tipo", "str", "Sendo CERTIFICADOS DE RECEBIVEIS DO AGRONEGOCIO = CRA; Sendo CERTIFICADOS DE RECEBIVEIS IMOBILIARIOS = CRI; Sendo DEBENTURES = DEB; Sendo CERTIFICADOS DE RECEBIVEIS = CR; Retorne se CRI, CRA, CR ou DEB. Caso não encontre, retorne ''."),
#     ("numero_emissao", "str", "Número da emissão da Securitizadora. Retorne apenas a forma numérica."),
#     ("numero_serie", "str", "Número da série da emissão. Retorne apenas a forma numérica. Caso seja uma série única, retorne UNI. Caso encontre mais de uma, retorne separados por ','."),
#     ("nome_securitizadora_emissora", "str", "Nome ou empresa ou sociedade cuja denominação ou significado está como Emissora ou Securitizadora. Caso não encontre retorne ''"),
#     ("nome_agente_fiduciario", "str", "Nome ou empresa ou sociedade cuja denominação ou significado está como Agente Fiduciário. Caso não encontre retorne ''"),
#     ("nome_devedora", "str", "Nome ou empresa ou sociedade cuja denominação ou significado está como Devedora ou que deve. Caso não encontre retorne ''. Caso encontre mais de um, retorne separados por ','"),
#     ("nome_avalista_fiadora", "str", "Nome ou empresa ou sociedade cuja denominação ou significado está como Avalista ou Fiador. Caso encontre mais de um, retorne separados por ','. Caso não encontre retorne ''"),
#     ("numero_avalista_fiadora", "str", "Número de inscrição no CNPJ do Avalista ou Fiador {nome_avalista_fiadora}. Caso encontre mais de um, retorne separados por ','. Caso não encontre retorne ''."),
#     ("numero_cnpj_securitizadora_emissora", "str", "Número da inscrição no Cadastro Nacional de Pessoa Jurídica (“CNPJ”) da Emissora ou Securitizadora - {nome_securitizadora_emissora}. Caso não encontre retorne ''"),
#     ("numero_cnpj_agente_fiduciario", "str", "Número da inscrição no Cadastro Nacional de Pessoa Jurídica (“CNPJ”) do Agente Fiduciário - {nome_agente_fiduciario}. Caso não encontre retorne ''"),
#     ("numero_cnpj_devedora", "str", "Número de inscrição no CNPJ da Devedora {nome_devedora}. Caso encontre mais de um, retorne separados por ','. Caso não encontre retorne ''."),
#     ("nome_cedente", "str", "Nome ou empresa ou sociedade cuja denominação ou significado está como Cedente(s) ou que cedeu. Caso encontre mais de um, retorne separados por ','. Caso não encontre retorne ''."),
#     ("numero_cpf_cnpj_cedente", "str", "Número de inscrição no CNPJ ou CPF do(s) Cedente(s). Caso encontre mais de um, retorne separados por ','. Caso não encontre retorne ''"),
#     ("numero_minimo_instalacao_assembleia_primeira_convocacao", "str", "Número mínimo de presentes para instalação de assembleia em primeira convocação. Caso não encontre retorne ''. Caso encontre, retorne o mais simples possível."),
#     ("numero_minimo_instalacao_assembleia_segunda_convocacao", "str", "Número mínimo de presentes para instalação de assembleia em segunda convocação. Caso não encontre retorne ''. Caso encontre, retorne o mais simples possível.")
# ]

In [22]:
campos = [
    ("tipo", "str", "Sendo CERTIFICADOS DE RECEBIVEIS DO AGRONEGOCIO = CRA; Sendo CERTIFICADOS DE RECEBIVEIS IMOBILIARIOS = CRI; Sendo DEBENTURES = DEB; Sendo CERTIFICADOS DE RECEBIVEIS = CR; Retorne se CRI, CRA, CR ou DEB. Caso não encontre, retorne ''."),
    ("numero_emissao", "str", "Número da emissão da Securitizadora. Retorne apenas a forma numérica."),
    ("numero_serie", "str", "Número da série da emissão. Retorne apenas a forma numérica. Caso seja uma série única, retorne UNI. Caso encontre mais de uma, retorne separados por ','."),
    ("nome_securitizadora_emissora", "str", "Nome ou empresa ou sociedade cuja denominação ou significado está como Emissora ou Securitizadora. Caso não encontre retorne ''"),
    ("nome_agente_fiduciario", "str", "Nome ou empresa ou sociedade cuja denominação ou significado está como Agente Fiduciário. Caso não encontre retorne ''"),
    # ("nome_devedora", "str", "Nome ou empresa ou sociedade cuja denominação ou significado está como Devedora ou que deve. Caso não encontre retorne ''. Caso encontre mais de um, retorne separados por ','"),
    # ("nome_avalista_fiadora", "str", "Nome ou empresa ou sociedade cuja denominação ou significado está como Avalista ou Fiador. Caso encontre mais de um, retorne separados por ','. Caso não encontre retorne ''"),
    # ("numero_avalista_fiadora", "str", "Número de inscrição no CNPJ do Avalista ou Fiador {nome_avalista_fiadora}. Caso encontre mais de um, retorne separados por ','. Caso não encontre retorne ''."),
    # ("numero_cnpj_securitizadora_emissora", "str", "Número da inscrição no Cadastro Nacional de Pessoa Jurídica (“CNPJ”) da Emissora ou Securitizadora - {nome_securitizadora_emissora}. Caso não encontre retorne ''"),
    # ("numero_cnpj_agente_fiduciario", "str", "Número da inscrição no Cadastro Nacional de Pessoa Jurídica (“CNPJ”) do Agente Fiduciário - {nome_agente_fiduciario}. Caso não encontre retorne ''"),
    # ("numero_cnpj_devedora", "str", "Número de inscrição no CNPJ da Devedora {nome_devedora}. Caso encontre mais de um, retorne separados por ','. Caso não encontre retorne ''."),
    # ("nome_cedente", "str", "Nome ou empresa ou sociedade cuja denominação ou significado está como Cedente(s) ou que cedeu. Caso encontre mais de um, retorne separados por ','. Caso não encontre retorne ''."),
    # ("numero_cpf_cnpj_cedente", "str", "Número de inscrição no CNPJ ou CPF do(s) Cedente(s). Caso encontre mais de um, retorne separados por ','. Caso não encontre retorne ''"),
    # ("numero_minimo_instalacao_assembleia_primeira_convocacao", "str", "Número mínimo de presentes para instalação de assembleia em primeira convocação. Caso não encontre retorne ''. Caso encontre, retorne o mais simples possível."),
    # ("numero_minimo_instalacao_assembleia_segunda_convocacao", "str", "Número mínimo de presentes para instalação de assembleia em segunda convocação. Caso não encontre retorne ''. Caso encontre, retorne o mais simples possível.")
]

In [44]:
campos = dict()
campos["nome_devedora"] = CamposLLM("nome_devedora", "str", "Nome ou empresa ou sociedade cuja denominação ou significado está como Devedora ou que deve. Caso não encontre retorne ''. Caso encontre mais de um, retorne separados por ','")
campos = dict()
campos["numero_cnpj_devedora"] = CamposLLM("numero_cnpj_devedora", "str", "Número de inscrição no CNPJ da Devedora {nome_devedora}. Caso encontre mais de um, retorne separados por ','. Caso não encontre retorne ''.")

In [ ]:
def get_campos_to_user_question(campos):
    campos_to_user_question = []
    
    for k, campo in campos.items():
        if campo.value == "" and campo.depends_on == []:
            campos_to_user_question.append(campo)
    
    return campos_to_user_question

In [23]:
output_parser, user_question = create_user_question(campos)

docs_relevants = get_docs_relevants(user_question)
print(docs_relevants)

output = invoke_llm(user_question, docs_relevants)
output


    Extraia as seguintes informações:
    
    tipo, numero_emissao, numero_serie, nome_securitizadora_emissora, nome_agente_fiduciario
    
    The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"tipo": str  // Sendo CERTIFICADOS DE RECEBIVEIS DO AGRONEGOCIO = CRA; Sendo CERTIFICADOS DE RECEBIVEIS IMOBILIARIOS = CRI; Sendo DEBENTURES = DEB; Sendo CERTIFICADOS DE RECEBIVEIS = CR; Retorne se CRI, CRA, CR ou DEB. Caso não encontre, retorne ''.
	"numero_emissao": str  // Número da emissão da Securitizadora. Retorne apenas a forma numérica.
	"numero_serie": str  // Número da série da emissão. Retorne apenas a forma numérica. Caso seja uma série única, retorne UNI. Caso encontre mais de uma, retorne separados por ','.
	"nome_securitizadora_emissora": str  // Nome ou empresa ou sociedade cuja denominação ou significado está como Emissora ou Securitizadora. Caso não encontre retorne ''
	"nome_age

'```json\n{\n\t"tipo": "CRA",\n\t"numero_emissao": "",\n\t"numero_serie": "",\n\t"nome_securitizadora_emissora": "",\n\t"nome_agente_fiduciario": ""\n}\n```'

In [ ]:
response_json = output_parser.parse(output)
response_json

In [40]:
campos_enabled = ['numero_minimo_instalacao_assembleia_primeira_convocacao', 'numero_minimo_instalacao_assembleia_segunda_convocacao']
campos_filtered = [c for c in campos if response_json[c[0]] == '' and c[0] in campos_enabled]

output_parser, user_question = create_user_question(campos_filtered)

docs_relevants = get_docs_relevants(user_question)

output = invoke_llm(user_question, docs_relevants)
response_json2 = output_parser.parse(output)
response_json2

{'numero_minimo_instalacao_assembleia_primeira_convocacao': '50% + 1',
 'numero_minimo_instalacao_assembleia_segunda_convocacao': 'qualquer número'}